### invae training

In [1]:
1

1

In [7]:
from inVAE import FinVAE, NFinVAE

ImportError: cannot import name 'inVAE' from 'inVAE' (/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/inVAE/__init__.py)

In [3]:
import warnings
import os
import sys
import gc
import warnings

import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import joblib
import numpy as np
import scipy.stats

adata_file_path = os.path.join("/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed", "dataset_complete_Suo.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path)
adata.X = adata.X.astype(np.float32)
print(adata_file_path, flush=True)
print(adata, flush=True)

def create_random_mask(shape, ratio_true, seed=None):
    rng = np.random.default_rng(seed)
    random_floats = rng.random(shape)
    mask = random_floats < ratio_true
    return mask

/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed/dataset_complete_Suo.h5ad
AnnData object with n_obs × n_vars = 841922 × 8192
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes'
    uns: 'rank_genes_groups'
    obsm: 'Unintegrated', 'X_pca', 'harmony'


In [4]:
print(f"CUDA: {torch.cuda.is_available()}", flush=True)

CUDA: False


In [5]:
adata_train_bool = create_random_mask(adata.shape[0], ratio_true=0.8, seed=0)
adata_train = adata[adata_train_bool].copy()
adata_valid = adata[~adata_train_bool].copy()

In [10]:
save_path = "/home/icb/kemal.inecik/lustre_workspace/tardis_data/models/invae_1.pt"

In [11]:
# Load saved weights and hyperparameters
saved_dict = torch.load(save_path, map_location = torch.device("cpu"))
model_state_dict = saved_dict['model_state_dict']
attr = saved_dict['attributes']
# Init the model
model = FinVAE(adata = adata, device = device, **attr)          
model.module.load_state_dict(model_state_dict)
# Put model in eval state (e.g. for batch-norm layers)
model.module.eval()

KeyError: 'model_state_dict'

In [14]:
saved_dict['attributes']

KeyError: 'attributes'

In [8]:
model = FinVAE.load(
    
    device = "cpu"
)

AttributeError: type object 'FinVAE' has no attribute 'load_model'

In [9]:
invariant = model.get_latent_representation(adata, latent_type='invariant')

Calculating latent representation of passed adata by trying to transfer setup from the adata the model was trained on!


/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arrs.append(arr[idx])
/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arrs.append(arr[idx])
/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future v

In [10]:
spurious = model.get_latent_representation(adata, latent_type='spurious')

Calculating latent representation of passed adata by trying to transfer setup from the adata the model was trained on!


/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arrs.append(arr[idx])
/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arrs.append(arr[idx])
/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future v

In [13]:
invariant.shape

(841922, 9)

In [14]:
spurious.shape

(841922, 1)

In [16]:
joblib.dump(invariant, "/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed/inveriant.joblib")
joblib.dump(spurious, "/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed/spurious.joblib")

['/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed/spurious.joblib']

In [21]:
a = joblib.load("/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed/inveriant.joblib")
b = joblib.load("/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed/spurious.joblib")

In [22]:
a

array([[ 1.6351789 ,  0.21473171, -1.192861  , ..., -0.36038458,
        -0.8183836 ,  0.2914055 ],
       [ 0.7872124 ,  0.07463333,  1.6079642 , ..., -0.61626107,
        -0.03099874, -1.2699432 ],
       [ 0.4563352 ,  0.15015881, -0.14327045, ..., -0.06415468,
        -1.907887  , -0.16704145],
       ...,
       [ 0.47114274, -0.8212458 , -0.04396553, ...,  0.09544043,
         0.5473186 , -0.19      ],
       [-0.49919507, -1.1582355 , -0.19781184, ..., -0.88468754,
         0.24713337,  0.4063162 ],
       [ 1.0077268 , -0.9353548 ,  0.5519111 , ..., -0.49778685,
         0.3943866 ,  1.5031106 ]], dtype=float32)